# TXT_to_XML-TEI personnalisé avec métadonnées pré-remplies
* En entrée : fichier .txt concaténé et pré-encodé de l'océrisation
* En sortie : un fichier xml avec TEI header pré-rempli (dont métadonnées du texte : titre, date de publi, lieu de publi, geonames, url du pdf, url de la notice BM, vers/prose...)
* Données nécessaires : tableurs Avancée_travail_corpus et ListeMazarinades en tsv (séparateur : tabulation)
* Le mode d'emploi du script se trouve dans le dernière cellule ("Exemple d'utilisation").

/ ! \ l'encodage automatique des fichiers se fait par alignement du nom du fichier avec l'id équivalent dans les tableurs. De fait, ce script ne fonctionne pas sur des fichiers dont le nom n'est pas strictement l'id Moreau,Labadie ou Socard (ex : Moreau85-2_GBOOKS ne marchera pas. En revanche, Moreau85_GBOOKS est ok.)

/ ! \ Ce script suit des process différents selon que les fichiers à encoder proviennent de GBOOKS, Gallica ou la BM. Chaque session d'encodage auto doit donc se faire sur des fichiers provenant de la même source. Cette source est à préciser en paramètre de la fonction XML_generate().

In [31]:
#Import des librairies nécessaires
import xml.etree.ElementTree as ET
import glob
import os
import re
import pandas as pd

In [34]:
def XML_generate(fichier_corpus="non_classé",genre="non renseigné",date_creation_fichier="3 juin 2021",
                 when_creation_fichier="2021-06-03",name_respXML="Roblin Camille",status_respXML="Stagiaire",
                 path_avancee_corpus="Avancée_travail_corpus - Corpus_complet.tsv", 
                 path_ListeMaz="ListeMazarinades - Documents_all.tsv", source_doc="GALL"):
                #Pour créer la structure XML-TEI + pré-remplir certains champs 
                #fichier_corpus : Ecrire le corpus dont font partie les documents (mazarinades_lettres, mazarinades_narration, 
                #mazarinades_chantables, sans indication). Il ne doit pas y avoir d'espace.
                #genre : genre du texte qui apparaitra dans les mots clés (lettre, chantable...)
                #date_creation_fichier : Ecrire  la date de lancement du script selon le format "3 juin 2021"
                #when_creation_fichier : Ecrire  la date de lancement du script selon le format "2021-06-03"
                #name_respXML et status_respXML : nom, prénom et statut de la personne responsable de la chaine de traitement TXT>XML
                #path_avancee_corpus : path vers la feuille du tableur Avancée_travail_corpus
                #path_ListeMaz : path vers la feuille du tableur ListeMazarinades
                #Source_doc : source des fichiers originaux ("GALL", "GBOOKS" ou "MAZ")
    
#Création de la structure XML-TEI générale
    
    #Création de l'élément racine, en l'occurence "TEI"
    root=ET.Element('TEI')
    root.set('xmlns','http://www.tei-c.org/ns/1.0')
    root.set('type',fichier_corpus)
    #Création du TEI Header
    teiHeader=ET.SubElement(root,'teiHeader')
    #Création du contenu de fileDesc
    fileDesc=ET.SubElement(teiHeader,'fileDesc') #Dès qu'on crée un sous-élément
    titleStmt=ET.SubElement(fileDesc,'titleStmt')
    title1=ET.SubElement(titleStmt,'title')
    title2=ET.SubElement(titleStmt,'title')
    title1.set('type','main') #On ajoute les attributs des balises
    title1.set('source','Moreau')
    title2.set('type','sub')
    title2.text="ANTONOMAZ édition" #On remplit la balise
    #Contenu de respStmt
    respStmt=ET.SubElement(titleStmt,'respStmt')
    resp=ET.SubElement(respStmt,"resp")
    resp.text="Chaîne de traitement"
    #Création et remplissage des balises pour responsable encodage XML-TEI
    persName_equipe_1=ET.SubElement(respStmt,'persName')
    persName_equipe_1.set('role','XML_Encoding')
    persName_equipe_1.text= name_respXML
    affiliation_equipe_1=ET.SubElement(persName_equipe_1,'affiliation')
    roleName_equipe_1=ET.SubElement(affiliation_equipe_1,"roleName")
    roleName_equipe_1.set('type','status')
    roleName_equipe_1.text=status_respXML
    #Création et remplissage des balises pour les chefs de projet
    #Chef projet 1
    persName_equipe_3=ET.SubElement(respStmt,'persName')
    persName_equipe_3.set('role','Project_manager')
    persName_equipe_3.set('ref','0000-0001-9518-1040')
    persName_equipe_3.text="Abiven Karine"
    #Chef projet 2
    persName_equipe_4=ET.SubElement(respStmt,'persName')
    persName_equipe_4.set('role','Project_manager')
    persName_equipe_4.set('ref','0000-0002-4795-2362')
    persName_equipe_4.text="Lejeune Gaël"
    #Création et remplissage des balises pour la personne responsable de l'OCRisation
    persName_equipe_5=ET.SubElement(respStmt,'persName')
    persName_equipe_5.set('role','OCRisation')
    persName_equipe_5.text="Tanguy Jean-Baptiste"
    persName_equipe_5.set('ref','0000-0002-0007-1664')
    #Création du publicationStmt
    publicationStmt=ET.SubElement(fileDesc,'publicationStmt')
    publisher=ET.SubElement(publicationStmt,'publisher')
    publisher.text="Projet ANTONOMAZ / Sorbonne Université"
    date_crea=ET.SubElement(publicationStmt,'date')
    date_crea.set('type','file_creation')
    date_crea.text=date_creation_fichier
    date_crea.set('when',when_creation_fichier)
    availability=ET.SubElement(publicationStmt,'availability')
    availability.set('status','restricted')
    availability.set('n','cc-by')
    licence=ET.SubElement(availability,'licence')
    licence.set('target','https://creativecommons.org/licenses/by/4.0')
    #Création du sourceDesc
    sourceDesc=ET.SubElement(fileDesc,'sourceDesc')
    bibl=ET.SubElement(sourceDesc,'bibl')
    ref=ET.SubElement(bibl,'ref')
    author=ET.SubElement(bibl,'author')
    persName_author=ET.SubElement(author,'persName')
    forename_author=ET.SubElement(persName_author,'forename')
    surname_author=ET.SubElement(persName_author,'surname')
    title3=ET.SubElement(bibl,'title')
    title3.set("source","Moreau")
    pubPlace=ET.SubElement(bibl,'pubPlace')
    pubPlace.set('ref','geonames:0000000')
    pubPlace.set('source','Moreau')
    publisher_imp=ET.SubElement(bibl,'publisher')
    publisher_imp.set('ref','isni:0000000')
    persName_pub=ET.SubElement(publisher_imp,'persName')
    forename_pub=ET.SubElement(persName_pub, 'forename')
    surname_pub=ET.SubElement(persName_pub,'surname')
    date_pub=ET.SubElement(bibl,'date')
    date_pub.set('source','Moreau')
    extent=ET.SubElement(bibl,'extent')
    measure=ET.SubElement(extent,'measure')
    measure.set('unit','page')
    note_format=ET.SubElement(bibl,'note')
    note_format.set('type','format')
    note_Moreau=ET.SubElement(bibl,'note')
    note_Moreau.set('type','Moreau_identifier')
    #note_Labadie=ET.SubElement(bibl,'note')
    #note_Labadie.set('type','Labadie_identifier')
    #note_Socard=ET.SubElement(bibl,'note')
    #note_Socard.set('type','Socard_identifier')
    note_BM=ET.SubElement(bibl,'note')
    note_BM.set('type','BM_identifier')
    note_BM.set('cert','low')
    ref_BM=ET.SubElement(bibl,'ref')
    ref_BM.set('type','BM_notice')
    ref_BM.set('cert','low')
    #Création du msDesc
    msDesc=ET.SubElement(sourceDesc,'msDesc')
    msIdentifier=ET.SubElement(msDesc,'msIdentifier')
    settlement=ET.SubElement(msIdentifier,'settlement')
    institution=ET.SubElement(msIdentifier,'institution')
    repository=ET.SubElement(msIdentifier,'repository')
    idno=ET.SubElement(msIdentifier,'idno')
    idno.set('type','cote')
    history=ET.SubElement(msDesc,'history')
    provenance=ET.SubElement(history,'provenance')
    stamp=ET.SubElement(provenance,'stamp')
    stamp.text="True/False"
    #Création de l'encodingDesc
    encodingDesc=ET.SubElement(teiHeader,'encodingDesc')
    projectDesc=ET.SubElement(encodingDesc,'projectDesc')
    p_proj_1=ET.SubElement(projectDesc,'p')
    p_proj_1.text="Cette édition a été réalisée dans le cadre du projet ANTONOMAZ. Son objectif principal est de fournir un texte destiné à l'exploration avec des outils électroniques. De ce fait, ce n'est ni une édition philologique, ni une édition pédagogique ou de redécouverte d'un auteur oublié."
    p_proj_2=ET.SubElement(projectDesc,'p')
    p_proj_2.text='Les textes encodés dans le cadre du projet ANTONOMAZ sont issus de numérisations de bibliothèques numériques publiques et de Google livres.'
    p_proj_3=ET.SubElement(projectDesc,'p')
    p_proj_3.text="L'édition présentée ici est issue d'un processus d'OCRisation réalisé avec Kraken."
    editorialDecl=ET.SubElement(encodingDesc,'editorialDecl')
    p_editorialDecl=ET.SubElement(editorialDecl,'p')
    #Création de du profileDesc
    profileDesc=ET.SubElement(teiHeader,'profileDesc')
    langUsage=ET.SubElement(profileDesc,'langUsage')
    language=ET.SubElement(langUsage,'language')
    language.set('ident','fr')
    language.text='Document en français'
    abstract=ET.SubElement(profileDesc,'abstract')
    p_abs=ET.SubElement(abstract,'p')
    p_abs.set("source","BM_notes")
    textClass=ET.SubElement(profileDesc,'textClass')
    keywords=ET.SubElement(textClass,'keywords')
    term_for_1=ET.SubElement(keywords,'term')
    term_for_1.set('type','form')
    term_for_1.text="vers/prose"
    term_for_2=ET.SubElement(keywords,'term')
    term_for_2.set('type','genre')
    term_for_2.text=genre
    term_for_3=ET.SubElement(keywords,'term')
    term_for_3.set('type','subgenre')
    term_for_4=ET.SubElement(keywords,'term')
    term_for_4.set('type','subgenre')
    term_for_5=ET.SubElement(keywords,'term')
    term_for_5.set('type','subject')
    #Création du corps du document : "text" et "body"
    text=ET.SubElement(root,'text')
    body=ET.SubElement(text,'body')
    
#Enregistrement du XML pour chacun des fichier .txt présent au même niveau que le script
    try:
        os.mkdir('sortie_xml')
    except:
        pass
    for filepath in glob.iglob('*.txt'):
        id_moreau=re.sub(".txt","",filepath)
        id_moreau=re.sub("_GALL|_GBOOKS|_MAZ","",id_moreau)
        print(id_moreau)
        #Ajout des métadonnées dans le TEI header à partir du Excel "Documents_All" exporté en TSV
        df=pd.read_csv(path_ListeMaz, sep='\t', header=0)
        p=re.compile(id_moreau+"$")#le $ sert à ce qu'un Moreau120 ne matche pas aussi le Moreau1201, Moreau1202 etc.
        for i in range(len(df)):
            #alignement du fichier avec les métadonnées du tableur
            m=p.match(str(df.loc[i,"id"]))
            if m:
                title1.text=str(df.loc[i,"titre"])
                title3.text=str(df.loc[i,"titre"])
                lieu_publi=str(df.loc[i,"lieu"])
                #remplissage auto du geonames en fonction du nom de la ville dans le tableur
                pubPlace.text=lieu_publi
                villes={"Paris":'geonames:2988507',
                        "Bâle":'geonames:2661604',
                        "Orléans":'geonames:2989317',
                        "Bordeaux":'geonames:6455058',
                        "Rotterdam":'geonames:2747891',
                        "Sedan":'geonames:2975349',
                        "Pontoise":'geonames:2986140',
                        "Embrun":'geonames:3020251',
                        "Saint-Germain-En-Laye":'geonames:2979783',
                        "Amsterdam (Paris)":'geonames:2988507',
                        "Bruxelles":'geonames:2802247',
                        "Rouen":'geonames:2982652',
                        "Cologne":'geonames:2886242',
                        "Lyon":'geonames:2996944',
                        "La Haye":'geonames:2747373',
                        "Troyes":'geonames:2971549',
                        "Torino":'geonames:3165524'}
                for cle, valeur in villes.items():
                    if lieu_publi==cle:
                        pubPlace.set('ref',valeur)
                #Remplissage auto des autres métadonnées
                date_pub.text=str(df.loc[i,"date_imprimée"])
                date_pub.set('when',str(df.loc[i,"date_imprimée"]))
                measure.set('quantity',str(df.loc[i,"nb pages"]))
                #Remplissage du note type = id_moreau
                idMoreau=str(df.loc[i,"id"])
                p2=re.compile("Moreau")
                m2=p2.match(idMoreau)
                #si le texte est identifié par Moreau
                if m2:
                    idMoreau_nb=re.sub("Moreau","",idMoreau)
                    note_Moreau.text=idMoreau_nb
                #si le texte a un autre identifiant (labadie, socard...)
                else:
                    note_Moreau.text="Sans ID Moreau"
                print("Remplissage auto des métadonnées à partir du tableur ListeMaz: OK.")
        #Ajout des métadonnées dans le TEI header à partir du Excel "Avancee_corpus" exporté en TSV
        df=pd.read_csv(path_avancee_corpus, sep='\t', header=0)
        p=re.compile(id_moreau+"$")
        for i in range(len(df)):
            m=p.match(str(df.loc[i,"id"]))
            if m:
                if source_doc=="GALL":
                    ref.set('target',str(df.loc[i,"PDF_GALLICA"]))
                elif source_doc=="GBOOKS":
                    ref.set('target',str(df.loc[i,"PDF_GBOOKS"]))
                elif source_doc=="MAZ":
                    ref.set('target',str(df.loc[i,"PDF_MAZ"]))
                ref_BM.set('target',str(df.loc[i,"Notice BM"]))
                vers_prose=str(df.loc[i,"Vers / Prose (corpus)"])
                if vers_prose=="V":
                    term_for_1.text="vers" #Vers ou Prose
                elif vers_prose=="P":
                    term_for_1.text="prose"
                elif vers_prose=="VP":
                    term_for_1.text="vers/prose"
                else:
                    term_for_1.text="non renseigné"             
                print("Remplissage auto des métadonnées à partir du tableur Avancee_corpus : OK.")
        #Ajout du texte concaténé et encodé dans le body du XML
        with open(filepath, "r",encoding="utf-8")as f:
            texte_encode=f.read()
        body.text=texte_encode
        #Remplissage auto de l'xml:id
        root.set('xml:id',id_moreau+"_"+source_doc)
        tree = ET.ElementTree(root)
        #Export du XML complet et pré-rempli
        tree.write("sortie_xml/%s_%s.xml" %(id_moreau,source_doc), encoding="utf-8",xml_declaration=False)
        #Correction des erreurs générées pendant la fusion txt + xml
        with open("sortie_xml/%s_%s.xml" %(id_moreau,source_doc), 'r', encoding="utf-8") as f:
            xmlstring=f.read()
        xmlstring_corrig = re.sub("&lt;", "<",xmlstring)
        xmlstring_corrig = re.sub("&gt;", ">",xmlstring_corrig)
        xmlstring_corrig = re.sub("&#10;", "",xmlstring_corrig)
        xmlstring_corrig = re.sub("^","<?xml version='1.0' encoding='UTF-8'?><?xml-model href='https://raw.githubusercontent.com/Antonomaz/ODD/master/Schema/ODD_Antonomaz.rng' type='application/xml' schematypens='http://relaxng.org/ns/structure/1.0'?> <?xml-model href='https://raw.githubusercontent.com/Antonomaz/ODD/master/Schema/ODD_Antonomaz.rng' type='application/xml' schematypens='http://purl.oclc.org/dsdl/schematron'?>",xmlstring_corrig)
        #Ecriture du fichier finalisé
        with open("sortie_xml/%s_%s.xml" %(id_moreau,source_doc), "w",encoding="utf-8")as f:
            fichier_final=f.write(xmlstring_corrig)
        print("Transformation TXT>XML-TEI terminée.")

# Exemple d'utilisation :
* Pour générer les XML, déposer ce script au même niveau que les fichiers .txt concaténés et pré-encodés (ex format nom : Moreau85_GALL.txt)
* Personnaliser les paramètres de la fonction XML_Generate() ci-dessous (la documentation pour savoir comment remplir les paramètres se trouve dans la cellule ci-dessus, en description de la fonction).
* Cliquer sur Cell > Run All.
* Si aucun message d'erreur n'apparait, les fichiers XML se trouvent dans le dossier "sortie_xml".

In [33]:
XML_generate(fichier_corpus="non_classé",genre="non renseigné",date_creation_fichier="3 juin 2021",
                 when_creation_fichier="2021-06-03",name_respXML="Roblin Camille",status_respXML="Stagiaire",
                 path_avancee_corpus="Avancée_travail_corpus - Corpus_complet.tsv",
             path_ListeMaz="ListeMazarinades - Documents_all.tsv", source_doc="GALL")

Moreau2273
Remplissage auto des métadonnées à partir du tableur ListeMaz: OK.
Remplissage auto des métadonnées à partir du tableur Avancee_corpus : OK.
Transformation TXT>XML-TEI terminée.
Moreau2412
Remplissage auto des métadonnées à partir du tableur ListeMaz: OK.
Remplissage auto des métadonnées à partir du tableur Avancee_corpus : OK.
Transformation TXT>XML-TEI terminée.
Moreau3004
Remplissage auto des métadonnées à partir du tableur ListeMaz: OK.
Remplissage auto des métadonnées à partir du tableur Avancee_corpus : OK.
Transformation TXT>XML-TEI terminée.
